# Imports

In [55]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path="langchain.db")
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain

from os.path import isfile, join
from os import listdir
import fitz
import docx
import pandas as pd

import openai
import os


# Create Index

In [66]:
basepath = '/Users/rorymclean/notebooks/ccqas/rfi/'

pdffiles = [f for f in listdir(basepath) if isfile(join(basepath, f)) and f.lower().endswith('.pdf')]
print(pdffiles)
docxfiles = [f for f in listdir(basepath) if isfile(join(basepath, f)) and f.lower().endswith('.docx')]
print(docxfiles)
txtfiles = [f for f in listdir(basepath) if isfile(join(basepath, f)) and f.lower().endswith('.txt')]
print(txtfiles)

[]
['CP_HRM_Solution_Special_Notice_RFI_FINAL.1690396353841.docx', 'Medical Quallity Assurance System RFI Attachment 1 Future Interface List.1690396361267.docx']
[]


In [67]:
mydata_all = ''

for z in pdffiles:
    fname = join(basepath,z)
    doc = fitz.open(fname)  
    this_doc = ''
    for page in doc:  # iterate the document pages
        this_doc = this_doc + page.get_text() + "\n"
    mydata_all = mydata_all + this_doc + '\n'  

for z in docxfiles: 
    fname = join(basepath,z)
    doc = docx.Document(fname)
    all_paras = doc.paragraphs
    this_doc = ''
    for para in all_paras:
        this_doc = this_doc+ para.text + "\n"
    mydata_all = mydata_all + this_doc + '\n'  
    
for z in txtfiles:
    fname = join(basepath,z)
    data_buffer = ""
    data_str_wrapper = open(fname,"r")
    data_buffer += data_str_wrapper.read()  
    mydata_all = mydata_all + data_buffer + '\n'     

    
print("Your document contains "+str(len(mydata_all))+" characters")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 0,
    length_function = len,
)
texts = text_splitter.create_documents([mydata_all])

print("You have "+str(len(texts))+" chunks")

Your document contains 16074 characters
You have 23 chunks


In [68]:
db3 = Chroma.from_documents(texts, OpenAIEmbeddings())


# Create Prompts

In [243]:
template = """
You are a helpful assistant that will use the TEXT below to answer the user's question.

TEXT:
{mytext}

"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "QUESTION: {myquestion}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])



# Ask Questions

In [244]:
def ask_question(myquestion,myrelevancy = .5, myk = 15, mymodel = 'gpt-3.5-turbo', mycache=True):

    docs = db3.similarity_search_with_relevance_scores(myquestion, k=myk)

    mytext = ""
    for x,v in docs:
        if mytext == "":
            mytext = str(x) + "\n\n"
            continue
        if v > myrelevancy:
            mytext = mytext + str(x) + "\n\n"

    # print(mytext)
    
    chain = LLMChain(
        llm=ChatOpenAI(model=mymodel,cache=mycache),
        prompt=chat_prompt,
    )
    
    results = chain.run(myquestion=myquestion, mytext=mytext)
    
    return results

In [98]:
# df_analysis = pd.DataFrame(columns=['question','chain','response',])

In [246]:
myquestion = "Does the document limit the number of pages in our response?"

results = ask_question(myquestion,myrelevancy=.4,myk=25,mycache=False)
print(results)


Number of requested results 25 is greater than number of elements in index 23, updating n_results = 23


Yes, the document specifies that the responses shall be no longer than 20 pages total.


In [99]:
myquestion = "Guess at the NAME or TITLE of this project"

results = ask_question(myquestion,myrelevancy=.1,myk=25,mycache=False)
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

Number of requested results 25 is greater than number of elements in index 23, updating n_results = 23


Title: Military Health System (MHS) Medical Quality Assurance System (MQAS)


In [102]:
myquestion = "Which agencies or departments are listed in this document?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The document mentions the Defense Health Agency (DHA) as the main agency responsible for the management of hospitals and clinics. It also references the National Defense Authorization Act (NDAA) and the Federal Acquisition Regulation (FAR) as regulatory frameworks. Additionally, there are mentions of the Joint Commission (TJC), the Military Health System (MHS), and the Department of Veterans Affairs (VA).


In [103]:
myquestion = "What function does his application perform?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The application performs the function of standardizing and centralizing the medical staff management capabilities of credentialing and privileging, as well as incorporating healthcare risk management for the Defense Health Agency (DHA) Military Treatment Facilities (MTFs) and the Services' operational environment. It also includes functions related to clinical quality management, document management, data storage and retrieval, collaboration tools, notifications and alerts, data analysis and reporting, user access control, and release of information.


In [104]:
myquestion = "Who are the users of this application?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The users of this application are not explicitly mentioned in the provided text. However, based on the context, it can be inferred that the users would include organizations within the Defense Health Agency (DHA), such as DHA MTFs (Medical Treatment Facilities), Combatant Commands, Services, Operational Units, VA (Veterans Affairs), Medical Centers, and Clinics. Additionally, the application may be used by providers, beneficiaries, and healthcare staff involved in credentialing, privileging, healthcare risk management, and clinical quality management processes.


In [105]:
myquestion = "What technology is referenced in the document and how are they used in this product?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The document references several technologies and their usage in the product. Some of the technologies mentioned include:

1. Application Programming Interfaces (API) Platform: This technology is used to provide seamless connection and interoperability with Solution Delivery Division (SDD) APIs and internal/external data exchanges.

2. Content management technology: This technology is used to automate the classification, storage, and retrieval of a large volume of C&P and HRM documentation.

3. Collaboration tools: These tools are used to facilitate collaboration among users, including notifications/alerts and access log files for information sharing.

4. Cloud hosted software: The product requires a high availability, Government FedRAMP+ Impact Level 4 approved Cloud hosted software for operational availability and cybersecurity compliance.

5. Credentialing, Privileging, and Healthcare Risk Management Business System Solution: The product aims to provide a scalable and flexible cloud-

In [106]:
myquestion = "Does the document reference cloud computing? Does the document recommend using cloud computing?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

Yes, the document does reference cloud computing. It states that industry partners should consider the capability to provide a Government FedRAMP+ Impact Level 4 approved Cloud hosted software with evidence of cybersecurity compliance or the ability to obtain the DHA Authority to Operate. However, it does not explicitly recommend using cloud computing.


In [107]:
myquestion = "Does the document reference COTS (commercial off the shelf) software? Does the document recommend using COTS software?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

Yes, the document does reference COTS (commercial off the shelf) software. It mentions that innovative approaches, including COTS, development efforts, and/or hybrid solutions, can be used to replace the current IT system, Centralized Credentials Quality and Assurance System (CCQAS). It encourages partnering to identify and leverage existing MHS solutions, such as Defense Medical Human Resource System-internet (DMHRSi), Joint Patient Safety Reporting (JPSR), and MHS GENESIS, to provide optimization, cost savings, and modernization for DHA Clinical Quality Management.


In [108]:
myquestion = "Does the document describe the current system architecture?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

No, the document does not describe the current system architecture. It provides information about the desired capabilities and functionalities for a new system or suite of systems that will replace the current system.


In [109]:
myquestion = "Does the document describe future changes they would like to see in new releases?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

No, the document does not describe future changes they would like to see in new releases. It primarily focuses on the capabilities and functionality sought by the Defense Health Agency (DHA) for the Medical Quality Assurance System (MQAS) and the requirements for potential vendors.


In [110]:
myquestion = "Assuming we provide a written response to this document, provide a document outline for our response?"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

Based on the provided text, here is a suggested document outline for your response:

I. Introduction
   A. Purpose of the Response
   B. Overview of your Company/Team/Solution Ownership

II. Contract and Teaming Strategy
   A. Description of Contract Strategy
   B. Teaming Strategy and Partnerships
   C. Business Type (NAICS, Hub Zone, Small/Large)
   D. Facility for Cleared Activities

III. Point of Contact Information
   A. Contact Information for POCs
   B. Phone, Address, Fax, Mail

IV. Resource Staffing Support
   A. Staffing Approach for Continuous Improvement Delivery
   B. Agile Sprint Methodologies

V. Responses to Cardinal Questions
   A. Detailed Responses to the Three Cardinal Questions
   B. Explanation of Solutions and Capabilities

VI. Models and Diagrams
   A. Physical and Logical Models
   B. Workflows and Solution Diagrams
   C. Systems Description Sections (as needed)

VII. Proactive Monitoring and Support Costs
   A. Proactive Monitoring Approach
   B. After-Hours a

In [112]:
df_analysis.to_csv(basepath+'ccqas.csv', index=False)

## Additional Questions for the Planner

In [115]:
myquestion = "In terms of capabilities and functionality sought by DHA, but do they mean by 'Minimum Outcomes' and what specific outcomes are they expecting? Provide a detailed answer."

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The DHA (Defense Health Agency) is seeking capabilities and functionality from industry partners that meet certain minimum outcomes. These outcomes represent the desired results or goals that the DHA expects the proposed solutions to achieve. The specific outcomes may vary depending on the area or domain being addressed. Here are some examples of the minimum outcomes mentioned in the provided text:

1. High Availability: The DHA expects a minimum operational availability of 98.5% for the proposed solution. This means that the system should be highly reliable and accessible to users.

2. Government FedRAMP+ Impact Level 4 approval: The DHA requires the proposed cloud-hosted software to be approved at the FedRAMP+ Impact Level 4. This certification ensures that the solution meets the necessary cybersecurity requirements.

3. Evidence of Cybersecurity Compliance: The industry partners should provide evidence of their cybersecurity compliance or demonstrate their ability to obtain the DHA 

In [117]:
myquestion = "Describe in detail the capabilities for privileged and Non-privileged Provider Credentialing"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The capabilities for privileged and non-privileged provider credentialing include the ability to collect and manage data related to provider credentials, as well as the ability to make informed decisions and provide support throughout the credentialing process. These capabilities are essential for ensuring that providers meet the necessary qualifications and standards to deliver quality healthcare.

1. Data Collection and Management: The system should be capable of collecting and storing relevant data regarding provider credentials, such as education, training, certifications, licenses, and work experience. It should also be able to capture and manage supporting documentation, such as transcripts, certificates, and letters of recommendation.

2. Decision Support Process Workflows: The system should provide customizable workflows that guide the credentialing process. These workflows should include automated checks and validation steps to ensure that all necessary information is collecte

In [119]:
myquestion = "Describe in detail the Capabilities required for Provider Privileging Decision Support"

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

The capabilities required for Provider Privileging Decision Support include the following:

1. Privileged and Non-privileged Provider Credentialing Data Collection and Record Management: The system should be capable of collecting and managing data related to both privileged and non-privileged providers. This includes capturing provider credentials, qualifications, training, certifications, licensure, and other relevant information.

2. Decision Support Process Workflows: The system should provide automated workflows that guide the decision-making process for provider privileging. This includes step-by-step processes for reviewing and evaluating provider qualifications, conducting background checks, verifying references, and assessing competency.

3. Reporting and Dashboard Capability: The system should have robust reporting and dashboard features that allow users to generate comprehensive reports and visualize data related to provider privileging. This includes tracking the status of p

In [121]:
myquestion = "In detail describe how does this document define 'Realistic Technical Capabilities'."

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

This document does not explicitly define 'Realistic Technical Capabilities'. However, it provides guidance for industry partners to consider each of the minimum outcomes listed when proposing a response. If any of the outcomes seem unrealistic, the document encourages industry partners to be candid and transparent in expressing their concerns. The document also mentions that technical architecture diagrams are preferred when possible. It suggests that industry partners should provide their capabilities for delivering a scalable and flexible cloud-based solution, obtaining cybersecurity accreditation and certification, and providing infrastructure, hardware, and software descriptions. Overall, the document emphasizes the need for innovative and cost-efficient solutions that align with the desired end-state and achievable objectives.


In [127]:
myquestion = "In detail describe how does this document defines required Function Capabilities."

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.1,myk=25,mycache=False
print(results)
df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Planner','response': results}

This document defines the required function capabilities by listing specific outcomes and functionalities that the integrated HRM and C&P system(s) should provide. These capabilities are outlined in various sections of the document. 

In the first section, it states that the system should have the capability to provide privileged and non-privileged provider credentialing data collection and record management. It should also support decision support process workflows and have reporting/dashboard capability on all data collected in the privileging process. 

The document also mentions the need for seamless connection to the Solution Delivery Division (SDD) Application Programming Interfaces (API) Platform or internal and external data exchanges. It emphasizes the importance of interoperability with MHS Systems and the need for content management technology to automate the classification, storage, and retrieval of a large volume of C&P and HRM documentation. Collaboration tools, notificat

In [245]:
df_analysis.to_csv(basepath+'ccqas.csv', index=False)

In [129]:
import string
import re
from IPython.display import display, Markdown, HTML

for index, row in df_analysis.iterrows():
    display(Markdown("<B>Question:</B>"), Markdown(row['question']))
    # display(Markdown("<B>Business Objective:</B>"), Markdown(row['O']))
    display(Markdown("<B>Response:</B>"), Markdown(row['response']))
    display(Markdown("---"))

<B>Question:</B>

Guess at the NAME or TITLE of this project

<B>Response:</B>

Title: Military Health System (MHS) Medical Quality Assurance System (MQAS)

---

...

<B>Question:</B>

In detail describe how does this document defines required Function Capabilities.

<B>Response:</B>

This document defines the required function capabilities by listing specific outcomes and functionalities that the integrated HRM and C&P system(s) should provide. These capabilities are outlined in various sections of the document. 

In the first section, it states that the system should have the capability to provide privileged and non-privileged provider credentialing data collection and record management. It should also support decision support process workflows and have reporting/dashboard capability on all data collected in the privileging process. 

The document also mentions the need for seamless connection to the Solution Delivery Division (SDD) Application Programming Interfaces (API) Platform or internal and external data exchanges. It emphasizes the importance of interoperability with MHS Systems and the need for content management technology to automate the classification, storage, and retrieval of a large volume of C&P and HRM documentation. Collaboration tools, notifications/alerts, and information access log files are also required capabilities.

Furthermore, the document highlights the need for resource staffing support that accounts for continuous improvement delivery using Agile Sprint methodologies. It also mentions the importance of physical and logical models, workflows, and solution diagrams, as well as systems description sections. The document asks for resource staffing life cycle by role and includes proactive monitoring, after-hours, and tier support costs models.

Additionally, the document specifies the capabilities required for recording, monitoring, and evaluating the privilege-specific competence of providers. It mentions the need for decision support workflow processes for clinical quality review of adverse events and adverse actions, as well as the release of information process workflow with reporting/dashboard capability. The system should also support impaired healthcare provider decision support workflow processes.

The document further outlines capabilities related to user account registration with multi-factor authentication, the ability to accommodate hierarchical relationships within and across complex organizations, and the provision of value-driven AI/Data Analytics information. It also emphasizes the need for high availability, Government FedRAMP+ Impact Level 4 approved Cloud hosted software with evidence of cybersecurity compliance.

In summary, this document defines the required function capabilities by specifying the outcomes and functionalities that the integrated HRM and C&P system(s) should provide, including data collection and management, decision support workflows, reporting and dashboard capabilities, interoperability, document management, collaboration tools, and cybersecurity compliance, among others.

---

# Create Chain

In [165]:
template = """
You are a helpful assistant that will search the internet to provide a detailed answer to the user's question. 
If the TEXT below helps answer the user's question, then incorporate it into your answer.
Otherwise, just use the search tool to find an answer.
Perform up to 8 internet searches if neccessary for answering the entire question.

TEXT:
{mytext}

"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = """
QUESTION: 
{myquestion}
Provide a very detailed answer to the user's question. Explain each element of your response.
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])



In [166]:
myprompt = chat_prompt.format_prompt(mytext="MyText stuff here", myquestion="Here is my question")
print(myprompt)

messages=[SystemMessage(content="\nYou are a helpful assistant that will search the internet to provide a detailed answer to the user's question. \nIf the TEXT below helps answer the user's question, then incorporate it into your answer.\nOtherwise, just use the search tool to find an answer.\nPerform up to 8 internet searches if neccessary for answering the entire question.\n\nTEXT:\nMyText stuff here\n\n", additional_kwargs={}), HumanMessage(content="\nQUESTION: \nHere is my question\nProvide a very detailed answer to the user's question. Explain each element of your response.\n", additional_kwargs={}, example=False)]


In [202]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

def ask_question(myquestion,myrelevancy = .7, myk = 15, mymodel = 'gpt-4', mycache=True, myverbosity=True):

    docs = db3.similarity_search_with_relevance_scores(myquestion, k=myk)

    mytext = ""
    for x,v in docs:
        if mytext == "":
            mytext = str(x) + "\n\n"
            continue
        if v > myrelevancy:
            mytext = mytext + str(x) + "\n\n"

    myprompt = chat_prompt.format_prompt(mytext=mytext, myquestion=myquestion)

    llm = ChatOpenAI(model=mymodel,cache=mycache,temperature=.2, n=2)
    search_agent = initialize_agent(
        tools=[DuckDuckGoSearchRun(name="Search")],
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        verbose=myverbosity,
        return_intermediate_steps=False,
    )
    
    results = search_agent.run(myprompt)
    
    return results

def save_to_dataframe(myquestion, mychain, myresponse):
    if len(df_analysis[df_analysis['question'] == myquestion]) > 0:
        myindex = df_analysis[df_analysis['question'] == myquestion].index.to_list()[0]
    else:
        myindex = len(df_analysis)

    df_analysis.loc[myindex]= {'question': myquestion, 'chain': mychain, 'response': myresponse}
    
    return

In [168]:
myquestion = "Describe the key steps when performing medical credentialing."

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.5,myk=25,mycache=False
print(results)

df_analysis.loc[len(df_analysis.index)]= {'question': myquestion, 'chain': 'Executer','response': results}



> Entering new AgentExecutor chain...
The user is asking for a detailed explanation of the key steps involved in medical credentialing. The provided text does not give a step-by-step process for medical credentialing, so I will need to use the search tool to find a detailed answer. 
Action: Search
Action Input: Key steps in medical credentialing process
Observation: In general, any licensed, independent healthcare professional who has been permitted by law and regulated by a licensing organization to provide services and care without supervision or direction within the scope of the individual's license needs to be credentialed. While every state has unique laws regarding medical practice, simply having a ... Provider credentialing is the process of assessing and establishing the provider's competency and verifying the practitioner's qualifications to practice medicine. It is a tedious process carried out by insurance companies. They verify the provider's qualifications to check if th

In [169]:
myquestion = "Are there any open source applications that perform medical credentialing."

results = ask_question(myquestion) #,mymodel="gpt-4",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)



> Entering new AgentExecutor chain...
The user is asking for open source applications that perform medical credentialing. To answer this question, I will need to search for open source applications that are used for medical credentialing.
Action: Search
Action Input: Open source applications for medical credentialing
Observation: MEDICAL CREDENTIALING SOFTWARE. The solution is a win-win for providers, patients, and facilities! The credentialing process is fast and hassle-free. With our credentialing software providers can start treating patients sooner, revenue is boosted, administrative work is reduced, and there is more time for patient care. Schedule a Demo. Health care credentialing software automates and simplifies the process of vendor, facility, and insurance provider credentialing. To stay compliant with health care facility policies and requirements, credentialing software takes over the manual process of verifying, screening, and generating contracts, applications, and othe

Thought:The search results still do not provide any specific examples of open-source applications that are used for medical credentialing. It seems that while there are open-source applications in the healthcare sector, such as Open Hospital, Open EMR, and Open MRS, these are primarily used for managing patient records. Medical credentialing appears to be a specialized function that may not have specific open-source applications. 
Final Answer: Based on the search results, there do not appear to be specific open-source applications designed for medical credentialing. While there are open-source applications in the healthcare sector, such as Open Hospital, Open EMR, and Open MRS, these are primarily used for managing patient records and improving healthcare delivery in low-resource environments. Medical credentialing, which involves verifying the qualifications and practice history of healthcare providers, appears to be a specialized function that may not have specific open-source appli

In [204]:
myquestion = "What kinds of decisions would require you to use medical credentialing information?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

Medical credentialing information is crucial for a variety of decisions in the healthcare sector. Here are some examples:

1. Provider Privileging: Medical credentialing information is used to evaluate the privilege-specific competence of a healthcare provider. This involves assessing whether the provider has documented evidence of competently performing the requested privilege(s) at the organization. This process is compliant with the Focused Professional Practice Evaluation (FPPE) and Ongoing Professional Practice Evaluation (OPPE) standards set by The Joint Commission (TJC).

2. Healthcare Risk Management: Medical credentialing information is used in decision-making processes related to healthcare risk management. This includes reviewing adverse events and actions, other reportable actions, and Potential Compensatory Events. The goal is to manage and mitigate risks associated with healthcare delivery.

3. Release of Information: When it comes to releasing information related to HRM 

In [206]:
myquestion = "How would a healthcare risk management system use medical credentialing information?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

A healthcare risk management system and medical credentialing information are both crucial components of a comprehensive healthcare management system. The Defense Health Agency (DHA) is working on integrating these two systems to improve the quality and safety of healthcare services.

Medical credentialing is a process that verifies the qualifications and practice history of a healthcare provider. This includes checking the provider's education, training, residency, and licenses, as well as any certifications issued by a board in the provider's specialty. The credentialing process ensures that healthcare providers have the necessary qualifications and experience to provide quality care to patients.

On the other hand, healthcare risk management is a process that identifies and assesses potential risks in a healthcare setting, with the goal of improving patient safety and quality of care, and reducing medical errors. This involves analyzing data from various sources, including medical c

In [225]:
myquestion = "What are unique metrics you would need to monitor when monitoring a medical credentialing system."

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

In the context of a medical credentialing system, the following metrics could be considered:

1. Total Days in Enrollment: This measures the average time it takes to enroll a provider in the system. A shorter enrollment time could indicate a more efficient system.

2. Application Processing Time: This measures how long it takes to process an application for credentialing. A shorter processing time could indicate a more efficient system.

3. Provider Performance: Continuous monitoring of provider performance and adherence to quality metrics is crucial. This could include measures of patient satisfaction, patient outcomes, and adherence to clinical guidelines.

4. Data Privacy and Security: Given the sensitive nature of the data involved in medical credentialing, metrics related to data privacy and security could be important. This could include measures of data breaches or other security incidents.

5. Compliance with Regulations: Compliance with relevant regulations and standards (such

In [227]:
myquestion = "Give a detailed description of the Healthcare Risk Management process. Describe in detail the metrics, processes, and users."

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

The Healthcare Risk Management (HRM) process is a systematic approach used in the management of clinical quality review of adverse events and adverse actions, other reportable actions, and release of information as appropriate. The overarching goal of HRM is to provide and maintain high-quality healthcare services. 

The HRM process involves several steps. First, it requires the ability to record, monitor, and evaluate the privilege-specific competence of the provider that lacks documented evidence of competently performing the requested privilege(s) at the organization. This is done through the Focused Professional Practice Evaluation (FPPE) and Ongoing Professional Practice Evaluation (OPPE) process, which is compliant with The Joint Commission (TJC).

Second, the HRM process involves decision support workflow processes for clinical quality review of adverse events and adverse actions, other reportable actions, and Potential Compensatory Events. This includes the ability to provide a

In [235]:
myquestion = "How does the \"Resource Management\" process benefit from an employee credentialing process?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

The employee credentialing process can significantly benefit the "Resource Management" process in several ways:

1. **Improving Trust**: Credentialing helps to establish trust within the organization and with external stakeholders. When employees are properly credentialed, it signifies that they have met certain standards of competence and ethical behavior. This can improve trust among team members, between employees and management, and with clients or customers who interact with the organization.

2. **Enhancing Revenue Cycle Management**: In a healthcare context, credentialing is a crucial part of the revenue cycle management process. It helps to avoid challenges like delays in payer enrollment, billing denials, and administrative errors. By ensuring that all healthcare providers are properly credentialed, the organization can reduce these issues and improve its financial performance.

3. **Increasing the Talent Pool**: Credentialing can attract more qualified candidates to the organ

In [237]:
myquestion = "How does the use of low-code software such as Appian benefit the customer?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

Low-code software like Appian offers several benefits to customers. Firstly, it allows for the development of software and processes visually through graphical interfaces and visual modeling. This approach can speed up the development process significantly, turning ideas into business apps up to 20 times faster than traditional coding. This speed allows businesses to quickly adapt to new opportunities. 

Additionally, low-code platforms enable the creation of consistent customer experiences across various platforms, including web, social, and mobile. This consistency can enhance the user experience and increase customer satisfaction.

Specifically, Appian software offers unique benefits. It helps end the culture of data decentralization, enabling better tracking of company processes. It provides interactive reports for business data analysis and centralizes communication with users. Appian also simplifies system integrations by providing a single point of access to all data sources, el

In [238]:
myquestion = "How does development with low-code software impace the Agile and DevOps process?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

Low-code development significantly impacts Agile and DevOps processes in several ways. Firstly, it allows for faster builds without compromising on power and flexibility. This is particularly noticeable even for high-performing, mature Agile teams who often find the increased development speed surprising when they first start building with low-code. Secondly, low-code supports the Agile approach of welcoming change. It does this through small teams that work in close collaboration, a microservices-based architecture that focuses effort, and instant cloud deployment to implement changes quickly. Thirdly, low-code platforms provide a visual development environment, as opposed to a traditional code-based one. This allows both development team members and business users to easily engage in the creation and review of functionality, provide feedback, validate assumptions, and identify improvements, thereby evolving the application in a more efficient and collaborative manner. Lastly, in the 

In [239]:
myquestion = "How might you benefit from the use of AI in the medical credentialling process?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

The use of Artificial Intelligence (AI) in the medical credentialing process can provide several benefits. 

1. Streamlining Verification: Traditional methods of verifying a healthcare professional's qualifications can be time-consuming and less precise. AI can streamline this process by quickly and accurately verifying credentials, saving time and resources.

2. Expediting Provider Onboarding: The onboarding process for new providers can be lengthy and complex. AI can expedite this process by automating certain tasks, such as background checks and verification of qualifications, reducing the time it takes to onboard new providers.

3. Ensuring Compliance: AI can help ensure compliance with regulatory standards by automatically checking for any discrepancies or issues that may violate these standards. This can help prevent potential legal issues and ensure that all providers are properly credentialed.

4. Promoting Workflow Efficiency: AI can automate many of the repetitive tasks invol

In [240]:
myquestion = "If another system accesses to a medical credentialling system what kind of data would they want to retrieve?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

The medical credentialing system mentioned in the text has several functions and capabilities that could provide valuable data to another system. These include:

1. Credentialing and Privileging (C&P) Data: The system collects and manages data related to the credentialing and privileging of healthcare providers. This data could be used by another system to verify the qualifications and privileges of a provider.

2. Healthcare Risk Management (HRM) Data: The system also manages data related to healthcare risk management. This includes data on malpractice claims, adverse events associated with healthcare, and administrative and criminal actions associated with healthcare providers. Another system could use this data to assess the risk associated with a particular provider or healthcare practice.

3. User Account Data: The system is capable of managing user accounts for over 200,000 users, with multi-factor authentication. Another system might want to retrieve this data to verify the iden

In [241]:
myquestion = "What kind of decisions would a decision support system serve when connected to a medical credentialing system or a medical risk management system?"

results = ask_question(myquestion, myverbosity=False) #,mymodel="gpt-3.5-turbo",myrelevancy=.5,myk=25,mycache=False
print(results)

save_to_dataframe (myquestion=myquestion, mychain='Executer',myresponse= results)

A decision support system (DSS) connected to a medical credentialing system or a medical risk management system would assist in making a wide range of decisions. 

For a medical credentialing system, the DSS would assist in decisions related to:

1. Provider Credentialing Data Collection and Record Management: The DSS would help in making decisions about the collection and management of data related to the credentials of healthcare providers. This includes decisions about what data to collect, how to manage it, and how to use it for decision-making.

2. Provider Privileging: The DSS would assist in making decisions about the privileging process for healthcare providers. This includes decisions about who should be granted privileges, what those privileges should be, and how to monitor and evaluate the performance of privileged providers.

For a medical risk management system, the DSS would assist in decisions related to:

1. Healthcare Risk Management: The DSS would help in making decis

In [242]:
df_analysis.head(100)

,question,chain,response
0,Guess at the NAME or TITLE of this project,Planner,Title: Military Health System (MHS) Medical Qu...
1,Which agencies or departments are listed in th...,Planner,The document mentions the Defense Health Agenc...
2,What function does his application perform?,Planner,The application performs the function of stand...
3,Who are the users of this application?,Planner,The users of this application are not explicit...
4,What technology is referenced in the document ...,Planner,The document references several technologies a...
5,Does the document reference cloud computing? D...,Planner,"Yes, the document does reference cloud computi..."
6,Does the document reference COTS (commercial o...,Planner,"Yes, the document does reference COTS (commerc..."
7,Does the document describe the current system ...,Planner,"No, the document does not describe the current..."
8,Does the document describe future changes they...,Planner,"No, the document does not describe future chan..."
9,Assuming we provide a written response to this...,Planner,"Based on the provided text, here is a suggeste..."


In [255]:
pdf_df = df_analysis.sort_values(by='chain',ascending=False)

from fpdf import FPDF

# class PDF(FPDF):
#     def header(self):
#         # Logo
#         self.image('logo_pb.png', 10, 8, 33)
#         # Arial bold 15
#         self.set_font('Arial', 'B', 15)
#         # Move to the right
#         self.cell(80)
#         # Title
#         self.cell(30, 10, 'Title', 1, 0, 'C')
#         # Line break
#         self.ln(20)

#     # Page footer
#     def footer(self):
#         # Position at 1.5 cm from bottom
#         self.set_y(-15)
#         # Arial italic 8
#         self.set_font('Arial', 'I', 8)
#         # Page number
#         self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

# Instantiation of inherited class
pdf = FPDF()
pdf.alias_nb_pages()
pdf.add_page()
pdf.set_font('Arial', 'BU', 16)
pdf.cell(0, 10, "ChatGPT Q&A's about CCQAS", 0, 1, 'C')

newchain = ''

for index, row in pdf_df.iterrows():
    if newchain != row['chain']:
        if newchain != "":
            pdf.add_page()
        pdf.set_text_color(r=0, g=0, b=255)
        pdf.set_font('Arial', 'B', 13)
        pdf.multi_cell(0, 10, "Chain Engine: "+row['chain'], 0, 1)
        newchain = row['chain']
        pdf.set_text_color(r=0, g=0, b=0)
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 6, 'Question:', 0, 1)
    pdf.set_font('Times', 'I', 11)
    pdf.multi_cell(0, 5, row['question'], 0, 1)
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 10, 'Response:', 0, 1)
    pdf.set_font('Times', '', 11)
    pdf.multi_cell(0, 5, row['response'], 0, 1)
    pdf.cell(0, 10, '_'*90, 0, 1)
    
pdf.output('ccqas_chatgpt.pdf', 'F')


''

In [233]:
df_analysis.drop(21, inplace=True)